## Small Example: gymnasium

Define action and state spaces. State spaces will be a dictionary of discrete places. We'll start with a discrete grid and then move to an openstreet map. 



In [ ]:
import gym
from gym import spaces
import numpy as np

class CitizenHealthMDP(gym.Env):
    def __init__(self, agent_profile=None, environment_context=None):
        super(CitizenHealthMDP, self).__init__()

        self.states = {"sick": 0, "healthier": 1, "sicker": 2}
        self.actions = {"receive_medical_attention": 0, "keep_forward": 1}
        self.action_space = spaces.Discrete(len(self.actions))
        self.observation_space = spaces.Discrete(len(self.states))

        self.agent_profile = agent_profile if agent_profile else {
            "health_state": 0,  # Scale from 0 (very sick) to 4 (very healthy)
            "housing_state": "ETHOS_0",  # ETHOS 0, 1, 2
            "administrative_state": "registered"  # "registered" or "non-registered"
        }
        
        # Transition probabilities and rewards (flexible for expansion)
        self.transition_probs = {
            (self.states["sick"], self.actions["receive_medical_attention"]): (
                1.0 if self.agent_profile["administrative_state"] == "registered" else 0.0,
                self.states["healthier"],
                +10
            ),
            (self.states["sick"], self.actions["keep_forward"]): (1.0, self.states["sicker"], -10),
        }
        
        impossible_actions = [
            action for state, action in self.transition_probs
            if self.transition_probs[(state, action)][0] == 0.0 and state == self.states["sick"]
        ]

        self.environment_context = environment_context if environment_context else {
            "shelters_available": 5,
            "government_healthcare_budget": 100000,
            "government_social_service_budget": 50000,
            "grid_position": (0, 0),  # (x, y) coordinates in the environment
            "deprived capabilities": impossible_actions
        }
        
        # Current state
        self.state = {"name": "sick", "index": self.states["sick"], "profile": agent_profile, "context": environment_context}
        
    def step(self, action):
        # Handle invalid actions gracefully
        state_index = self.state["index"]
        if (state_index, action) not in self.transition_probs:
            prob, next_state_index, reward = (0.0, state_index, 0)
        else:
            prob, next_state_index, reward = self.transition_probs[(state_index, action)]
        
        # State transition based on probability
        if np.random.rand() < prob:
            next_state_name = [name for name, idx in self.states.items() if idx == next_state_index][0]
            self.state["name"] = next_state_name
            self.state["index"] = next_state_index

            # Update health state according to the new state
            if next_state_name == "healthier":
                self.state["profile"]["health_state"] = min(4, self.state["profile"]["health_state"] + 1)
            elif next_state_name == "sicker":
                self.state["profile"]["health_state"] = max(0, self.state["profile"]["health_state"] - 1)
        
        
        # Check if episode is done
        done = self.state["index"] in [self.states["healthier"], self.states["sicker"]]
        
        return self.state, reward, done, {}
    
    def reset(self, seed=None, options=None):
        self.state["name"] = "sick"
        self.state["index"] = self.states["sick"]
        self.state["profile"] = self.agent_profile
        self.state["context"] = self.environment_context
        return self.state, {}
    
    def render(self):
        print(f"Current state: {self.state['name']}, Profile: {self.state['profile']}, Context: {self.state['context']}")


Current state: sick, Profile: {'health_state': 2, 'housing_state': 'ETHOS_0', 'administrative_state': 'registered'}, Context: {'shelters_available': 5, 'government_healthcare_budget': 100000, 'government_social_service_budget': 50000, 'grid_position': (0, 0)}
